# Machine Learning I, Programming Exercise 3

## Prerequisites
In this exercise, we will implement fundamental extensions to the support vector machine from last exercise. Copy the necessary code from programming exercise 2 to get the SVM running (with gradient descent and SGD) into the following cell. That means both the `fit(X, y)` and the `predict(X)` method should work correctly. If you didn't solve the last programming exercise, you can also copy the relevant parts from the sample solution that was uploaded to Olat.

## 1. Multi-class SVM
The support vector machines that we have implemented so far and also the theoretical model that was discussed in the lecture have one fundamental restriction: They can only distinguish between two classes and are therefore called *binary* classifiers. If we instead wish to assign a label to a data point from more than two possible choices, we call that a *multi-class* classification problem.

There are several models that extend the SVM so that it can deal with more than two classes, but in this exercise we are only going to have a look at one of the simplest possibilities: the *one-against-all* (OAA) approach. First, let $f: \mathcal{X} \to \mathbb{R}$ be the *decision function* of a two-class SVM, i.e.,
\begin{equation*}
    f(\mathbf{x}) = \mathbf{w}^T \mathbf{x} + b.
\end{equation*}
Then the predicted label of $\mathbf{x}$ is just $\operatorname{sign}(f(\mathbf{x}))$. Also assume that we want to assign each datapoint a label from the set $\mathcal{Y} = \{1, \ldots, c\}$. As hinted by the name *one-against-all*, we can now train $c$ different binary classifiers ($f_i$ for each $i \in \mathcal{Y}$) with a slightly modified dataset: For classifier $f_i$ all datapoints with original label $i$ are relabeled to $1$ and all others are changed to $-1$. This is then a binary classification problem, which we know how to solve. That leaves us with the problem of "combining" the output of the $c$ classifiers to make a reasonable prediction. We accomplish that by using the following decision rule:
\begin{equation*}
    f(\mathbf{x}) = \operatorname{arg\ max}_{i \in \mathcal{Y}} f_{i}(\mathbf{x})
\end{equation*}

Your tasks are as follows:
1. Extend your SVM binary implementation(s) with a method `decision_function(X)` that implements $f$ as detailed above.
2. Implement the OAA algorithm inside the `OAA_SVM` class given in the code skeleton.
3. (optional) If you are bored and have nothing better to do, you can also have a look at the *one-against-one* strategy. As the name suggests, one binary classifier is trained for each pair of possible class labels. The predicted label is chosen by running all binary classifiers and choosing the class that was predicted most often. To avoid ties you can also sum up the decision function values for each class instead (but be careful with the signs).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

class OAA_SVM(BaseEstimator, ClassifierMixin):
    def __init__(self, svm_class, *svm_args, **svm_kwargs):
        super(OAA_SVM, self).__init__()
        
        self.bin_svms = None
        self.svm_class = svm_class
        self.svm_args = svm_args
        self.svm_kwargs = svm_kwargs
        self.classes = None
    
    def fit(self, X, y):
        raise NotImplementedError('TODO')
        
    def predict(self, X):
        raise NotImplementedError('TODO')


# Generate toy data
n = 200
X, Y = make_classification(n, n_features=2, n_redundant=0, n_classes=3,
                           n_clusters_per_class=1, class_sep=1.3, random_state=654321)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=12345)

from sklearn.svm import LinearSVC
classifiers = [
    #('Gradient Descent', OAA_SVM(SVM_GD, C=1*n, bias=True, step_size=lambda t: 0.01/(t+1))),
    ('LinearSVC', LinearSVC(C=1))
]

# Plot decision surface
# First generate grid
res = 200  # Resolution of the grid in cells
x_max, y_max = np.max(X, axis=0) + 0.25
x_min, y_min = np.min(X, axis=0) - 0.25
grid_x, grid_y = np.meshgrid(np.linspace(x_min, x_max, res),
                             np.linspace(y_min, y_max, res))
# Get test array from grid
grid_input = np.c_[grid_x.reshape(-1), grid_y.reshape(-1)]

cmap = colors.ListedColormap([[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])
rows = (len(classifiers)+1)//2
fig, axes = plt.subplots(rows, 2, sharex=True, sharey=True, figsize=(12, 4.5*rows))
for (name, clf), ax in zip(classifiers, axes):
    clf.fit(X_train, Y_train)
    score = clf.score(X_test, Y_test)
    grid_out = clf.predict(grid_input).reshape(grid_x.shape)

    ax.set_title('Classifier: %s, Acc.: %.3f' % (name, score))
    ax.contourf(grid_x, grid_y, grid_out, alpha=0.5, cmap=plt.cm.brg)
    ax.scatter(X_train[:, 0], X_train[:, 1], c=Y_train, cmap=cmap, edgecolor='k')
plt.tight_layout()
plt.show()
plt.close(fig)

## 2. Kernel SVM
In this exercise, we want to exploit the kernel trick and turn our linear SVM into a non-linear learning algorithm. Given a dataset $D = \{(\mathbf{x}^{(1)}, y^{(1)}), (\mathbf{x}^{(2)}, y^{(2)}), \ldots, (\mathbf{x}^{(n)}, y^{(n)})\}$ and a kernel function $k: \mathbb{R}^d \times \mathbb{R}^d \to \mathbb{R}$, we can state the following optimization problem for the kernel SVM:
\begin{equation*}
    \min_{\boldsymbol{\alpha} \in \mathbb{R}^n, b \in \mathbb{R}} \frac{1}{2}  \boldsymbol{\alpha}^T K \boldsymbol{\alpha}  + \frac{C}{n} \sum_{i=1}^{n} \max\left(0, 1 - y^{(i)}\left(\sum_{j=1}^{n} \boldsymbol{\alpha}_j k(\mathbf{x}^{(i)}, \mathbf{x}^{(j)}) + b\right)\right),
\end{equation*}
where $K \in \mathbb{R}^{n \times n}$ is the so-called *kernel matrix* with $K_{ij} = k(\mathbf{x}^{(i)}, \mathbf{x}^{(j)})$.

The steps to implement the kernel SVM will be pretty much the same as for the linear SVM from the last sheet:
1. Write a function `kernel_svm_gradient` that computes the gradient of the objective function above w.r.t. $\boldsymbol{\alpha}$ and $b$.


2. Implement a number of batched kernel functions. Given two matrices $X \in \mathbb{R}^{n \times d}$ and $X' \in \mathbb{n' \times d}$, those functions should return a matrix $K \in \mathbb{R}^{n \times n'}$ with $K_{ij} = k(X_i, X'_j)$, where $k: \mathbb{R}^d \times \mathbb{R}^d \to \mathbb{R}$ is the kernel function in question:
    * The linear kernel: $k(\mathbf{x}, \mathbf{x}') = \langle \mathbf{x}, \mathbf{x}' \rangle$
    * The polynomial kernel: $k(\mathbf{x}, \mathbf{x}') = (\langle \mathbf{x}, \mathbf{x}' \rangle + c)^d$
    * The Gaussian (RBF) kernel: $k(\mathbf{x}, \mathbf{x}') = \exp\left(-\frac{1}{2\sigma^2} \lVert \mathbf{x} - \mathbf{x}' \rVert^2\right)$. **Hint:** $\lVert \mathbf{x} - \mathbf{x}' \rVert^2 = \lVert \mathbf{x} \rVert^2 - 2\langle \mathbf{x}, \mathbf{x}' \rangle + \lVert \mathbf{x}' \rVert^2$
    
   If you want to witness the results of your computations before dying of old age, I'd recommend *not* to use any `for` loops.


3. Implement the kernel SVM using (regular) gradient descent by filling in the gaps of the class `Kernel_SVM_GD`. Your implementation should accept one of the kernel functions from part 2 and use that to compute the kernel matrices necessary for training and prediction.


4. Repeat task 3, but this time use stochastic "gradient" descent to optimize the objective function. That is, choose a random batch $B \subset \{1, \ldots, n\}$ of a given size from the training data to compute the kernel matrix $K_{BB}$ (i.e., it should be restricted to only the data points in $B$) and update only the corresponding alphas.


5. Replace the Scikit-learn versions of the kernel SVM in the visualisation with your own and see if you can achieve similar results. You can then start plying around with kernel parameters to see what happens.

**Hint:** Don't be discouraged if you get very bad results at first, this is not necessarily due to a bug in your implementation. The kernel SVM is *very* sensitive to changes in the hyperparameters. Especially $\sigma$ for the RBF (Gaussian) kernel and the step size can have a huge influence on the end result. Try to log the value of the objective function during optimizaion to see if that explodes and experiment not just with constant step sizes, but try decaying step sizes as well (a step size in the order of $\mathcal{O}(1/t)$ is usually a good start). 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.datasets import make_classification, make_moons
from sklearn.model_selection import train_test_split

def kernel_svm_loss(alpha, b, C, K, y):
    # Instead of X, we are passing the kernel matrix K directly
    return 0.5*np.matmul(alpha.T, np.matmul(K, alpha)) + C/K.shape[0] * np.sum(hinge_loss(y*(np.matmul(K, alpha) + b)), axis=0)

def kernel_svm_gradient(alpha, b, C, K, y):
    raise NotImplementedError('TODO')

class Kernel_SVM_GD(BaseEstimator, ClassifierMixin):
    def __init__(self, kernel_func, bias=True, C=1, step_size=0.01, max_iter=1000):
        super(Kernel_SVM_GD, self).__init__()
        
        self.kernel_func = kernel_func
        self.bias = bias
        self.C = C
        self.step_size = step_size
        self.max_iter = max_iter
        self.alpha = None
        self.b = None
        self.X_train = None

    def fit(self, X, y):
        raise NotImplementedError('TODO')

    def predict(self, X):
        return np.sign(self.decision_function(X))
    
    def decision_function(self, X):
        raise NotImplementedError('TODO')
    
class Kernel_SVM_SGD(Kernel_SVM_GD):
    def __init__(self, kernel, batch_size, bias=True, C=1, step_size=0.01, max_iter=1000):
        super(Kernel_SVM_SGD, self).__init__(kernel, bias=bias, C=C, step_size=step_size, max_iter=max_iter)
        
        self.batch_size = batch_size

    def fit(self, X, y):
        raise NotImplementedError('TODO')


def linear_kernel(X1, X2):
    raise NotImplementedError('TODO')

def poly_kernel(c=0, d=1, gamma=1):
    def kernel(X1, X2):
        raise NotImplementedError('TODO')
    return kernel

def rbf_kernel(sigma=1):
    def kernel(X1, X2):
        raise NotImplementedError('TODO')
    return kernel

# Generate toy data
n = 100
X, Y = make_moons(n, noise=0.2, random_state=123456)
# Change labels from 0,1 to -1,1
Y *= 2; Y -= 1
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=12345)

# You can also try a dataset with more than 2 classes
#X, Y = make_classification(n, n_features=2, n_redundant=0, n_classes=3,
#                           n_clusters_per_class=1, class_sep=1.3, random_state=654321)
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=12345)


from sklearn.svm import SVC
classifiers = [
    # Use this to add your own classifiers to the visualisation
    ('Linear Kernel (sklearn)', SVC(C=1, kernel='linear')),
    ('Quadratic Kernel (sklearn)', SVC(C=1, kernel='poly', degree=2, gamma=0.88, coef0=0)),
    ('RBF Kernel (sklearn)', SVC(C=1, gamma=0.88))  # here, gamma = 1/(2*sigma**2)
]

# Plot decision surface
# First generate grid
res = 200  # Resolution of the grid in cells
x_max, y_max = np.max(X, axis=0) + 0.25
x_min, y_min = np.min(X, axis=0) - 0.25
grid_x, grid_y = np.meshgrid(np.linspace(x_min, x_max, res),
                             np.linspace(y_min, y_max, res))
# Get test array from grid
grid_input = np.c_[grid_x.reshape(-1), grid_y.reshape(-1)]

cmap = colors.ListedColormap([[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])
rows = (len(classifiers)+1)//2
fig, axes = plt.subplots(rows, 2, sharex=True, sharey=True, figsize=(12, 4.5*rows))
for (name, clf), ax in zip(classifiers, axes.ravel()):
    clf.fit(X_train, Y_train)
    score = clf.score(X_test, Y_test)
    grid_out = clf.predict(grid_input).reshape(grid_x.shape)

    ax.set_title('Classifier: %s, Acc.: %.3f' % (name, score))
    ax.contourf(grid_x, grid_y, grid_out, alpha=0.5, cmap=plt.cm.brg)
    ax.scatter(X_train[:, 0], X_train[:, 1], c=Y_train, cmap=cmap, edgecolor='k')
plt.tight_layout()
plt.show()
plt.close(fig)

## 3. Model Selection & Model Evaluation
Now that we have a working algorithm for the general kernel SVM, it is only natural to try it out on a somewhat more complex task than the two dimensional toy datasets that we have used up until now. However, we need to address two problems that naturally arise in  virtually any application of machine learning:
1. *Model evaluation*: This refers to the idea that we want to know how good or bad our model performs after we have trained it. Actually, we are interested in the performance (e.g. classification accuracy) that we can expect from a model when we apply it to real-world data it has never seen before. Ideally, we would have an infinite and highly available stream of real-world data which we could use to evaluate the model, but data is quite often a rare and precious resource in machine learning and we have to make the best out of what is available to us.

   Using the same data for evaluation that we have already used for training is clearly not a good idea, since that would only estimate the performance on already seen data. A popular solution to the problem is to randomly split the available data into a *training set* that is used for training the model and a *test set* that is held out from the training procedure and then used for evaluating the model. In general, this is not a bad idea and often used in practice, but with this approach, we could be unlucky and, by chance, the test set might not be very representative of the true underlying real-world data distribution. The point estimate we recieve could be way off in this case. An improvement to this method is *cross validation*, which you have already seen in the lecture. Here, we randomly split the data into $k$ folds of the same size before using one fold as the test set and the rest for training. We repeat this procedure $k$ times so that each fold has been used for testing exactly once and in the end we recieve a distribution for the performance of the model.


2. *Model selection*: Another problem that we need to address is that of *model selection*. This refers to the procedure of finding a good model for the underlying task. Two different models don't need to be prduced by different algorithms, instead the models that we get when running the SVM algorithm with $C=1$ and $C=10$, respectively, would be considered different in this context. In fact, model selection often coincides with hyperparameter search for a specific algorithm. A simple, yet often used, approach to this problem is called *grid search*. A priori, we define a list of possible values for each hyperparameter and then simply evaluate the performance of the resulting model for every possible combination of those hyperparameter values. Again, measuring the performance could be achieved using a train-test-split or cross validation, as detailed in 1. After we have tested all possible combinations, we simply pick the one with the highest estimated performance.


Now, the question that would naturally arise in this scenario is, whether this performance estimate for the best model is a good estimator of the model's performance on unseen data. Unfortunately, the answer is no, since we have used the held-out test data to make decisions about the hyperparameters of the model, meaning that we implicitely "leaked" information on the test data by choosing the hyperparameters that yield the best performance on it.

Clearly, we need to hold out another portion of the data from the model selection procedure, so that we can finally estimate the model's performance on this unseen subset. In the simplest case, we split the data into three subsets:
* The *test set*, which we use to estimate the model's performance after the whole procedure and
* The *training* and *validation set*, which are used for model selection.

As mentioned above, we could also use cross validation to estimate the performance of whatever comes out of the model selection process, resulting in what is usually called *nested cross validation*. The name stems from the observation that we use cross validation to estimate the model's performance in the "outer" loop and also during model selection in the "inner" loop. The following image shows how the data would be divided for $k=5$ in the outer loop and $k=2$ in the inner loop:

<figure>
  <img src="https://miro.medium.com/max/1400/1*8WAHvOZkLwtwFLwWPVNhow.png" style="padding: 5px; border: 2px solid #000; max-width:700px; max-height:700px;" />
  <figcaption style="text-align: center;">Data splits in nested cross validation (<a href="https://towardsdatascience.com/validating-your-machine-learning-model-25b4c8643fb7">source</a>).</figcaption>
</figure>

Another, more mathematically founded explanation can be found in a machine learning textbook of your choice, for example in "[Mathematics for Machine Learning](https://mml-book.github.io/book/mml-book.pdf)" by M. P. Deisenroth, A. A. Aldo Faisal, and C. S. Ong (see chapter 8.6, p. 284).

In this exercise, we will use nested cross validation to estimate the performance of our SVM implementstion on a somewhat more complex and difficult dataset.
1. Write a function `cross_validation` that takes the following arguments:
    * The classifier whose performance we want to estimate. It should implement both the `fit(X, y)` and the `score(X, y)`method.
    * Training data in the form of an `(X, y)` tuple.
    * The number of folds $k$.
   
   The function should implement "normal" cross validation and return the resulting scores as an array of length $k$.


2. Write a function `grid_search` that takes the following arguments:
    * An arbitrary function $f$, called the *objective*. We want to find the arguments that maximize this function.
    * A grid of function parameter values to test. This should be a dictionary that has the arguments of $f$ as its keys and a list of values that we want to try for those arguments as its values.
   
   Assume that the function is $f(a, b) = a^2 - 6b$ and that we receive the parameter dictionary `{'a': [1, 2, 3], 'b': [5, 6, 7]}`. Then we would plug in each possible combination of $a$ ad $b$ into $f$, i.e., $f(1, 5), f(1, 6), f(1, 7), f(2, 5), \ldots, f(3, 7)$. The function should return the parameters that maximized the objective function and the value of the objective function that is obtained when plugging in those parameters. In the example those would be $a=3, b=5$ and $f(3, 5) = -21$.


3. Implement a class `GridSearch` that emulates a classifier by training an underlying base classifier with grid search. Its constructor should take the following arguments:
    * A factory function that takes an arbitrary set of parameters and returns an instance of the base classifier with the specified parameters.
    * A parameter grid, on which the grid search should be performed. Should have the same format as for the gird search function from 2.
    
   Additionally, the class itself should implement the following methods:
    * `fit(X, y)`: Find the parameters from the given parameter grid that maximize the base classifier's performance by using the grid search function from 2. After that, train the base model using the best parameters and all available data.
    * `score(X, y)`: Here, you can simply pass the call to the trained base classifier.


4. Use all of the above to implement nested cross validation as a function. It should take the following arguments:
    * A factory function like in 3.
    * Training data in the form of an `(X, y)` tuple.
    * A grid of function parameter values to test, like in 2.
    * Two parameters `k_inner` and `k_outer` for setting the number of folds for the inner and the outer cross validation loop, respectively.


5. Finally, use nested cross validation to estimate the performance of your multi-class kernel SVM implementation on Scikit-Learn's [digits dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits). Use $k=5$ for both the inner and outer loop and standardize the data before passing it to the classifier. It is recommended that you stick to the RBF kernel and test different values for $\sigma$ and $C$ only. Try to keep the parameter grid relatively small so that the search doesn't take forever. You should be able achieve more than 90% mean cross validation accuracy.

**Hint:** Running the nested cross validation might be really slow, depending on your hardware. Try to report progress regularly so that you can see how far the process got and that you can estimate the remaining time. If you find that your implementation is way too slow, you can also take Scikit-Learn's `SVC` class as the underlying classifier instead.

In [ ]:
from sklearn.datasets import load_digits


def cross_validation(clf, X, Y, k=5):   
    raise NotImplementedError('TODO')

def grid_search(func, param_grid):
    raise NotImplementedError('TODO')

class GridSearch:
    def __init__(self, clf_factory, param_grid, cv_k=5):
        self.clf = None
        self.clf_factory = clf_factory
        self.param_grid = param_grid
        self.cv_k = cv_k

    def fit(self, X, y):
        raise NotImplementedError('TODO')

    def score(self, X, y):
        if self.clf is None:
            raise ValueError('Score can only be called after supplying training data with fit first!')
        
        return self.clf.score(X, y)

def nested_cross_validation(clf_factory, X, y, param_grid, outer_k=5, inner_k=5):   
    raise NotImplementedError('TODO')


digits = load_digits()
X, Y = digits.data, digits.target

param_grid = {
    'kernel': [rbf_kernel(gamma) for gamma in [1, 2]],
    'C': [1, 10],
}
# TODO: Perform nested cross validation with your SVM implementation here
